In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
import pandas as pd

In [2]:
# Method for executing and printing the query
def start_query():
    try:
        # start querie and conversion
        ret = sparql.queryAndConvert()

        # bring result to pandas df
        df = pd.json_normalize(ret["results"]["bindings"])

        # format df
        col_names=df.columns.values #get column names
        col_values = [] #create list for relevant columns
        for i in col_names:
            if "value" in i:   #filter for column names with value in the name (this are the relevant)
                col_values.append(i) #add to the created list
        sparql_result = df[col_values] #new df with just the relevant columns

        print(sparql_result)

    except:
        print("except")

In [29]:
sparql = SPARQLWrapper(
    "http://vm188067-l3s.hosting.uni-hannover.de:7200/repositories/Dsms"
)
sparql.setReturnFormat(JSON)

sparql.setQuery("""
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX matvoc: <http://stream-ontology.com/matvoc-core/>

        select ?material_name_instance ?material_name where { 
            ?material_name_instance rdf:type matvoc:Formula;
                    ?data_property ?material_name.
            ?data_property rdf:type owl:DatatypeProperty.
        } limit 5 
    """
)

start_query()

                        material_name_instance.value material_name.value
0  https://stream-projekt.net/ea2b3bdd5ced355cbcc...            AlSi10Mg
1  https://stream-projekt.net/ce82267cc22098ad4d2...            AlSi10Mg
2  https://stream-projekt.net/9f282f00ec2b1bab5f1...            AlSi10Mg
3  https://stream-projekt.net/9d844b98be3f07c0a10...            AlSi10Mg
4  https://stream-projekt.net/cb301f72272314352ef...            AlSi10Mg


# 1. Tensile Test

## 1.1 What is the measured "Maximum Tensile Strength" of the given material "CuCrZr"?

Querying for a MeasuredQuantitativeProperty of a specific Measurement (Filtered by known TestID or unique Material). All given values, no calculation or aggregation needed.

In [39]:
sparql = SPARQLWrapper(
    "http://vm188067-l3s.hosting.uni-hannover.de:7200/repositories/Dsms"
)
sparql.setReturnFormat(JSON)

sparql.setQuery("""
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX matvoc: <http://stream-ontology.com/matvoc-core/>
        PREFIX mech: <http://emmo.info/emmo/domain/mechanical-testing#>
        PREFIX emmo: <http://emmo.info/emmo#>
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

        select ?tensile_strength ?material_name where { 
        
        ?tensile_strength_instance rdf:type mech:EMMO_926d14c7_4f78_4c37_99b4_dfb0a531db64; 
            emmo:EMMO_8ef3cd6d_ae58_4a8d_9fc0_ad8f49015cd0 ?value_instance.
        ?value_instance emmo:EMMO_faf79f53_749d_40b2_807c_d34244c192f4 ?tensile_strength .
        
        ?measurement_instance emmo:hasMeasurementResult ?tensile_strength_instance ;
            emmo:hasProcessInput ?specimen_instance .
        
        ?specimen_instance emmo:EMMO_e1097637_70d2_4895_973f_2396f04fa204 ?material_name_instance .
        
        ?material_name_instance rdf:type mech:EMMO_71200c24_44fe_4533_96be_55ecb299681c ;
            emmo:EMMO_23b579e1_8088_45b5_9975_064014026c42 ?material_name .
        
        FILTER(?material_name = "AlSi10Mg")
        
        
        } 
        ORDER BY DESC(?tensile_strength) 
        LIMIT 1
    """
)

start_query()

  tensile_strength.value material_name.value
0               2.01e+02            AlSi10Mg


## 1.2 What is the maximum measured "Tensile Strength" of all "CuCrZr" samples?
Querying for a MeasuredQuantitativeProperty out of all tested specimens made of a specific Material. Aggregate for the maximum value.


Not to answer with the dataset

## 1.3 Which Material has the highest measured Value Rp02?
Querying for a MeasuredQuantitativeProperty out of all entrys in the database. Aggregate for the maximum value.

In [46]:
sparql = SPARQLWrapper(
    "http://vm188067-l3s.hosting.uni-hannover.de:7200/repositories/Dsms"
)
sparql.setReturnFormat(JSON)

sparql.setQuery("""
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX matvoc: <http://stream-ontology.com/matvoc-core/>
        PREFIX mech: <http://emmo.info/emmo/domain/mechanical-testing#>
        PREFIX emmo: <http://emmo.info/emmo#>
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

        select ?material_name ?rp02 where { 
        
        ?rp02_instance rdf:type mech:EMMO_44c9b071_54a1_4366_84e4_b14b055ec633; 
            emmo:EMMO_8ef3cd6d_ae58_4a8d_9fc0_ad8f49015cd0 ?value_instance.
        ?value_instance emmo:EMMO_faf79f53_749d_40b2_807c_d34244c192f4 ?rp02 .
        
        ?measurement_instance emmo:hasMeasurementResult ?rp02_instance ;
            emmo:hasProcessInput ?specimen_instance .
        
        ?specimen_instance emmo:EMMO_e1097637_70d2_4895_973f_2396f04fa204 ?material_name_instance .
        
        ?material_name_instance rdf:type mech:EMMO_71200c24_44fe_4533_96be_55ecb299681c ;
            emmo:EMMO_23b579e1_8088_45b5_9975_064014026c42 ?material_name .
        

        
        
        } 
        ORDER BY DESC(?rp02) 
        LIMIT 1
    """
)

start_query()

  rp02.value material_name.value
0   1.42e+02            AlSi10Mg


## 1.4 Which datasets that have "Tensile Strength" values between the range X and Y?
Querying for all Dataset with a MeasuredQuantitativeProperty in a specific range.

In [58]:
sparql = SPARQLWrapper(
    "http://vm188067-l3s.hosting.uni-hannover.de:7200/repositories/Dsms"
)
sparql.setReturnFormat(JSON)

sparql.setQuery("""
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX matvoc: <http://stream-ontology.com/matvoc-core/>
        PREFIX mech: <http://emmo.info/emmo/domain/mechanical-testing#>
        PREFIX emmo: <http://emmo.info/emmo#>
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        PREFIX tensile: <http://emmo.info/application/tensiletest-ontology#>

        select ?dataset_instance ?tensile_strength where { 
        
        ?dataset_instance rdf:type emmo:EMMO_194e367c_9783_4bf5_96d0_9ad597d48d9a ;
            emmo:EMMO_b2282816_b7a3_44c6_b2cb_3feff1ceb7fe ?tensile_strength_instance .
        
        ?tensile_strength_instance rdf:type mech:EMMO_926d14c7_4f78_4c37_99b4_dfb0a531db64; 
            emmo:EMMO_8ef3cd6d_ae58_4a8d_9fc0_ad8f49015cd0 ?value_instance.
        ?value_instance emmo:EMMO_faf79f53_749d_40b2_807c_d34244c192f4 ?tensile_strength .
        
        FILTER(?tensile_strength < 200 && ?tensile_strength > 180)
        
        } 
        LIMIT 10
    """
)

start_query()

                              dataset_instance.value tensile_strength.value
0  https://stream-projekt.net/f3fe5269379c6a72e03...               1.86e+02
1  https://stream-projekt.net/ce82267cc22098ad4d2...               1.87e+02
2  https://stream-projekt.net/4c8f6f3aed12f8cef76...               1.89e+02
3  https://stream-projekt.net/8c9132ab9e86d3ae617...               1.92e+02
4  https://stream-projekt.net/9d844b98be3f07c0a10...               1.94e+02
5  https://stream-projekt.net/d65e9f5966bd5ae1439...               1.94e+02
6  https://stream-projekt.net/cb301f72272314352ef...               1.96e+02
7  https://stream-projekt.net/f015c338bbb8162d903...               1.96e+02
8  https://stream-projekt.net/7b1e087edee81627af4...               1.98e+02
9  https://stream-projekt.net/ea2b3bdd5ced355cbcc...               1.99e+02


# 2. Hardness Test

## 2.1 Which datasets that have "Hardness" (Vickers/Brinell/Knoop/Rockwell) values between the range X and Y?
Querying for Datasets with MeasuredQuantitativeProperty of a specific ProcedureUnit between a specifc range

## 2.2 How many datasets of the material "X" have more than only one Hardness-Type (Vickers/Brinell/Knoop/Rockwell) values?
Count of Datasets of a specific EngineeredMaterial with more than one ProcedureUnit for Hardness


# 0 Gerneral Questions

## 0.1 Where are the "Data" for the "Test" with the given "ID: xyz"?
Querying for the StoragePath of a Dataset of a specific Measurement with the TestID "Test123"

## 0.2  Which Materials where tested yet? (with respect to the specific test type TENSILE TEST)
Querying a list of all EngineeredMaterials of a specific Measreuement Subclass

### Matvoc based

In [82]:
sparql = SPARQLWrapper(
    "http://vm188067-l3s.hosting.uni-hannover.de:7200/repositories/Dsms"
)
sparql.setReturnFormat(JSON)

sparql.setQuery("""

        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX matvoc: <http://stream-ontology.com/matvoc-core/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX oboe: <http://ecoinformatics.org/oboe/oboe.1.2/oboe-core.owl#>
        PREFIX emmo: <http://emmo.info/emmo#>
        
        select DISTINCT ?measurement_class_label where { 
           # ?material_name_instance rdf:type matvoc:Formula;
           #         ?data_property ?material_name.
           # ?data_property rdf:type owl:DatatypeProperty.
           
           ?measurement_class rdfs:subClassOf oboe:Measurement ;
               skos:prefLabel ?measurement_class_label .
           ?measurement_instance rdf:type ?measurement_class .
           
            FILTER (?measurement_class_label != "Measurement"@en )
        } limit 5 
    """
)

start_query()

  measurement_class_label.value
0                   TensileTest


## 0.3 Which datasets test a specimens that has the material X as part of their composition in the concentration range from Y% to Z%?
Querying a Dataset of a Specimen that is made of an Alloy with MassConcentration of a specific Material

## 0.4 Which datasets were conducted using machine X?
Querying a Dataset whose Measurement was conducted using a specific Machine "X"
